In [1]:
%autoreload 2

In [196]:
from argparse import Namespace
from collections import defaultdict
import copy
from datetime import datetime
import difflib
import duckdb
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML  # type: ignore
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tatsu, tatsu.ast
import tqdm.notebook as tqdmn


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../reward-machine'))

import compile_predicate_statistics
import compile_predicate_statistics_split_args
from compile_predicate_statistics_split_args import *
from config import SPECIFIC_NAMED_OBJECTS_BY_ROOM

import ast_printer
import ast_parser

In [43]:
cache_dir = compile_predicate_statistics.get_project_dir() + '/reward-machine/caches'

# regular_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics.pkl'))
split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_028b3733.pkl.gz'))
split_args_df = split_args_df[split_args_df['predicate'] != 'same_type']
print(split_args_df.shape)
# split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_4d5dd602.pkl.gz'))

# stats = compile_predicate_statistics.CommonSensePredicateStatistics(cache_dir)
split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(
    # cache_dir, compile_predicate_statistics_split_args.CURRENT_TEST_TRACE_NAMES, overwrite=False
    )

(625506, 8)


2023-08-07 15:20:11 - compile_predicate_statistics_split_args - INFO     - Loaded data with shape (2703800, 8) from /Users/guydavidson/projects/game-generation-modeling/reward-machine/caches/predicate_statistics_028b3733.pkl.gz


After filtering out same_type, data has shape (625506, 8)


In [323]:
split_args_df_no_same_type = split_args_df[split_args_df['predicate'] != 'same_type'].drop(columns=['string_intervals'])
split_args_df_no_same_type.shape

(625506, 8)

In [324]:
split_args_df_no_same_type.to_pickle(os.path.join(cache_dir, 'predicate_statistics_028b3733.pkl.gz'))

In [44]:
split_args_stats.trace_lengths_and_domains_df.select(pl.col('trace_length').max()).item()

35172

In [59]:
a = np.zeros(10000, dtype=np.uint8)
a[100:1000] = 1


'000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [65]:
t = np.zeros(10, dtype=np.uint8)
t[1:4] = 1
b = t.tobytes()
d = {b[0]: '0', b[1]: '1'}

In [68]:
''.join(map(lambda x: d[x], b))

'0111000000'

In [73]:
b = bytes([0, 1])
b

b'\x00\x01'

In [107]:
MAX_LENGTH = split_args_stats.trace_lengths_and_domains_df.select(pl.col('trace_length').max()).item()
print(MAX_LENGTH)
b = bytes([0, 1])
BYTE_MAPPING = {b: str(i) for i, b in enumerate(b)}



def intervals_to_string(intervals, max_length: int = MAX_LENGTH):
    value = np.zeros(max_length, dtype=np.uint8)
    for interval in intervals:
        value[interval[0]:interval[1]] = 1

    return ''.join(map(lambda b: BYTE_MAPPING[b], value.tobytes()))
    # return np.array2string(value, separator='', threshold=max_length + 10)[1:-1].replace('\n ', '')


string_intervals = split_args_df.intervals.apply(intervals_to_string)

# small_split_args_df = small_split_args_df.assign()

35172


In [108]:
split_args_df = split_args_df.assign(string_intervals=string_intervals)
split_args_df.head()

,predicate,arg_1_id,arg_1_type,arg_2_id,arg_2_type,trace_id,domain,intervals,string_intervals
0,adjacent,Shelf|-02.97|+01.53|-01.72,shelf,Shelf|-02.97|+01.53|-02.47,shelf,1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rereco...,medium,"[[0, 1612]]",1111111111111111111111111111111111111111111111...
1,adjacent,Shelf|-02.97|+01.53|-02.47,shelf,Shelf|-02.97|+01.53|-01.72,shelf,1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rereco...,medium,"[[0, 1612]]",1111111111111111111111111111111111111111111111...
2,adjacent,Shelf|-02.97|+01.53|-01.72,shelf,west_wall,wall,1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rereco...,medium,"[[0, 1612]]",1111111111111111111111111111111111111111111111...
3,adjacent,west_wall,wall,Shelf|-02.97|+01.53|-01.72,shelf,1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rereco...,medium,"[[0, 1612]]",1111111111111111111111111111111111111111111111...
4,adjacent,Shelf|+00.62|+01.01|-02.82,shelf,BridgeBlock|+01.03|+01.11|-02.88,bridge_block,1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rereco...,medium,"[[0, 1612]]",1111111111111111111111111111111111111111111111...


In [114]:
duckdb.sql('DROP TABLE data')
duckdb.sql('CREATE TABLE data AS SELECT * FROM split_args_df')
duckdb.sql('ALTER TABLE data ALTER string_intervals TYPE BITSTRING')
duckdb.sql('DESCRIBE data').show()

# duckdb.sql("CREATE TYPE predicate AS ENUM (SELECT predicate FROM split_args_df);")
# duckdb.sql("CREATE TYPE domain AS ENUM (SELECT domain FROM split_args_df);")
# duckdb.sql("CREATE TYPE trace_id AS ENUM (SELECT trace_id FROM split_args_df);")
# all_types = tuple([t for t in set(split_args_df.arg_1_type.unique()) | set(split_args_df.arg_2_type.unique()) if isinstance(t, str) ])
# duckdb.sql(f"CREATE TYPE arg_type AS ENUM {all_types};")
# all_ids = tuple([t for t in set(split_args_df.arg_1_id.unique()) | set(split_args_df.arg_2_id.unique()) if isinstance(t, str)])
# duckdb.sql(f"CREATE TYPE arg_id AS ENUM {all_ids};")

# duckdb.sql('ALTER TABLE data ALTER predicate TYPE predicate')
# duckdb.sql('ALTER TABLE data ALTER domain TYPE domain')
# duckdb.sql('ALTER TABLE data ALTER trace_id TYPE trace_id')
# duckdb.sql('ALTER TABLE data ALTER arg_1_type TYPE arg_type')
# duckdb.sql('ALTER TABLE data ALTER arg_2_type TYPE arg_type')
# duckdb.sql('ALTER TABLE data ALTER arg_1_id TYPE arg_id')
# duckdb.sql('ALTER TABLE data ALTER arg_2_id TYPE arg_id')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│   column_name    │ column_type │  null   │   key   │ default │ extra │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ predicate        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ arg_1_id         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ arg_1_type       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ arg_2_id         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ arg_2_type       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ trace_id         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ domain           │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ intervals        │ INTEGER[][] │ YES     │ NULL    │ NULL    │  NULL │
│ string_intervals │ BIT         │ YES     │ NULL    │ NULL    │  NULL │
└──────────────────┴─────────────┴─────────┴───────

In [167]:
tld_df = split_args_stats.trace_lengths_and_domains_df
duckdb.sql('DROP TABLE trace_lengths_domains')
duckdb.sql('CREATE TABLE trace_lengths_domains AS SELECT * FROM tld_df')
duckdb.sql('ALTER TABLE trace_lengths_domains ALTER domain TYPE domain')
duckdb.sql('ALTER TABLE trace_lengths_domains ALTER trace_id TYPE trace_id')
duckdb.sql('SELECT * FROM trace_lengths_domains').show()


┌────────────────────────────────────────────────────┬──────────────┬──────────┐
│                      trace_id                      │ trace_length │  domain  │
│                      trace_id                      │    int64     │ "domain" │
├────────────────────────────────────────────────────┼──────────────┼──────────┤
│ 1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rerecorded │         1612 │ medium   │
│ 1El1CmicSoKZKTLe8NpP-gameplay-attempt-2-rerecorded │           56 │ medium   │
│ 1El1CmicSoKZKTLe8NpP-preCreateGame-rerecorded      │         1250 │ medium   │
│ 1HOTuIZpRqk2u1nZI1v1-gameplay-attempt-1-rerecorded │          921 │ many     │
│ 1HOTuIZpRqk2u1nZI1v1-preCreateGame-rerecorded      │          550 │ many     │
│ 1aTng0m9240WEAU975WE-createGame-rerecorded         │          648 │ few      │
│ 1aTng0m9240WEAU975WE-gameplay-attempt-1-rerecorded │          984 │ few      │
│ 1aTng0m9240WEAU975WE-gameplay-attempt-2-rerecorded │         1387 │ few      │
│ 1aTng0m9240WEAU975WE-preCr

In [182]:
duckdb.sql("SELECT arg_1_type FROM data WHERE arg_1_type='ball'")

┌────────────┐
│ arg_1_type │
│  arg_type  │
├────────────┤
│   0 rows   │
└────────────┘

In [183]:
duckdb.sql("SELECT arg_1_id as '?b' FROM data WHERE arg_1_type = 'dodgeball' ")

┌────────────────────────────────┐
│               ?b               │
│             arg_id             │
├────────────────────────────────┤
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│ Dodgeball|-02.60|+00.13|-02.18 │
│               ·                │
│               ·                │
│               ·                │
│ Dodgeball|+00.70|+01.11|-02.80 │
│ Dodgeball|+00.19|+01.13|-02.80 │
│ Dodgeball|+00.19|+01.13|-02.80 │
│ Dodgeball|+00.19|+01.13|-02.80 │
│ Dodgeball|+00.19|+01.13|-02.80 │
│ Dodgeball|+00.19|+01.13|-02.80 │
│ Dodgeball|+00.19|+01.13|-02.80 │
│ Dodgeball|+00.70|+01.11|-02.80 │
│ Dodgeball|+00.70|+01.11|-02.80 │
│ Dodgeball|+00.70|+01.11|-02.80 │
├────────────────────────────────┤
│ ? rows (>9999 rows

BIT

In [496]:
TEMP_TABLE_PREFIX = "temp_table_"
DEBUG = True

class CommonSensePredicateStatisticsDatabse():
    data: pd.DataFrame
    domains: typing.List[str]
    predicates: typing.List[str]
    room_objects_cache: typing.Dict[str, typing.Set[str]]
    # same_type_arg_cache: typing.Dict[str, typing.List[typing.Tuple[str, str, str, str]]]
    trace_lengths_and_domains: typing.Dict[str, typing.Tuple[int, str]]
    trace_lengths_and_domains_df: pl.DataFrame


    def __init__(self,
                 cache_dir: typing.Union[str, pathlib.Path] = DEFAULT_CACHE_DIR,
                 trace_names: typing.Sequence[str] = FULL_PARTICIPANT_TRACE_SET,
                 cache_rules: typing.Optional[typing.Sequence[str]] = None,
                 base_trace_path: typing.Union[str, pathlib.Path] = DEFAULT_BASE_TRACE_PATH,
                 cache_filename_format: str = DEFAULT_CACHE_FILE_NAME_FORMAT,
                 trace_lengths_filename_format: str = DEFAULT_TRACE_LENGTHS_FILE_NAME_FORMAT,
                 in_progress_traces_filename_format: str = DEFAULT_IN_PROCESS_TRACES_FILE_NAME_FORMAT,
                 force_trace_names_hash: typing.Optional[str] = None,
                 temp_table_prefix: str = TEMP_TABLE_PREFIX,
                 overwrite: bool = False, trace_hash_n_characters: int = 8):
        
        self.object_assignment_cache = {}
        self.temp_table_index = 0
        self.predicates = split_args_stats.predicates
        self.trace_lengths_and_domains_df = split_args_stats.trace_lengths_and_domains_df
        self.domains = split_args_stats.domains
        self.max_length = self.trace_lengths_and_domains_df.select(pl.col('trace_length').max()).item()
        self.temp_table_prefix = temp_table_prefix

        # duckdb.create_function("empty_bitstring", self.create_empty_bitstring_function(self.max_length), [], duckdb.typing.BIT)


    def _predicate_key(self, predicate: str, args: typing.Sequence[str]) -> str:
        return f"{predicate}-({','.join(args)})"

    def _domain_key(self, domain: str):
        if domain.endswith('few_new_objects'):
            return 'few'
        elif domain.endswith('semi_sparse_new_objects'):
            return 'medium'
        elif domain.endswith('many_new_objects'):
            return 'many'
        else:
            raise ValueError(f"Unrecognized domain: {domain}")

    def _object_assignments(self, domain, variable_types, used_objects=[]):
        '''
        Wrapper around get_object_assignments in order to cache outputs
        '''

        key = (domain, tuple(variable_types), tuple(used_objects))
        if key not in self.object_assignment_cache:
            object_assignments = get_object_assignments(domain, variable_types, used_objects=used_objects)
            self.object_assignment_cache[key] = object_assignments

        return self.object_assignment_cache[key]

    
    def filter(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]]):
        temp_table_names = [t[0] for t in duckdb.sql('SHOW TABLES').fetchall() if t[0].startswith(self.temp_table_prefix)]
        for table_name in temp_table_names:
            duckdb.sql(f'DROP TABLE {table_name};')
        
        try:
            self.temp_table_index = 0
            outcome_table_name, _ = self._inner_filter(predicate, mapping)
            n_traces = duckdb.sql(f"SELECT COUNT(DISTINCT(trace_id)) FROM {outcome_table_name}").fetchone()[0]  # type: ignore
            n_intervals = duckdb.sql(f"SELECT COUNT(*) FROM {outcome_table_name}").fetchone()[0]  # type: ignore
            n_total_states = duckdb.sql(f"SELECT SUM(bit_count(string_intervals)) FROM {outcome_table_name}").fetchone()[0] # type: ignore
            return n_traces, n_intervals, n_total_states
            
        except PredicateNotImplementedException as e:
            # TODO: decide what we return in this case, or if we pass it through and let the feature handle it
            raise e
        
    def _table_name(self, index: int):
        return f"{self.temp_table_prefix}{index}"
    
    def _next_temp_table_index(self):
        self.temp_table_index += 1
        return self.temp_table_index
    
    def _next_temp_table_name(self):
        return self._table_name(self._next_temp_table_index())
    
    def create_empty_bitstring_function(self, length: int):
        def empty_bitstring():
            return '0' * length
        
        return empty_bitstring

    def _handle_predicate(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]]) -> typing.Tuple[pl.DataFrame, typing.Set[str]]:
        predicate_name = extract_predicate_function_name(predicate)  # type: ignore

        if predicate_name not in self.predicates:
            raise PredicateNotImplementedException(predicate_name)

        variables = extract_variables(predicate)  # type: ignore
        used_variables = set(variables)

        # Restrict the mapping to just the referenced variables and expand meta-types
        relevant_arg_mapping = {}
        for var in variables:
            if var in mapping:
                relevant_arg_mapping[var] = sum([META_TYPES.get(arg_type, [arg_type]) for arg_type in mapping[var]], [])

            # This handles variables which are referenced directly, like the desk and bed
            elif not var.startswith("?"):
                relevant_arg_mapping[var] = [var]

        select_items = ["trace_id", "domain", "string_intervals"]
        where_items = [f"predicate='{predicate_name}'"]
        
        for i, (arg_var, arg_types) in enumerate(relevant_arg_mapping.items()):
            # if it can be the generic object type, we filter for it specifically
            if GAME_OBJECT in arg_types:
                where_items.append(f"arg_{i + 1}_type NOT IN {tuple(GAME_OBJECT_EXCLUDED_TYPES)}")
                
            else:
                if len(arg_types) == 1:
                    where_items.append(f"arg_{i + 1}_type='{arg_types[0]}'")
                else:
                    where_items.append(f"arg_{i + 1}_type IN {tuple(arg_types)}")

            select_items.append(f"arg_{i + 1}_id as '{arg_var}'")

        table_name = self._next_temp_table_name()
        query = f"CREATE TEMP TABLE {table_name} AS SELECT {', '.join(select_items)} FROM data WHERE {' AND '.join(where_items)};"
        if DEBUG: print(query)
        duckdb.sql(query)
        return table_name, used_variables

    def _handle_and(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]]) -> typing.Tuple[pl.DataFrame, typing.Set[str]]:
        and_args = predicate["and_args"]
        if not isinstance(and_args, list):
            and_args = [and_args]

        table_names = []
        used_variables_by_child = []
        all_used_variables = set()

        for and_arg in and_args:  # type: ignore
            try:
                sub_table_name, sub_used_variables = self._inner_filter(and_arg, mapping)
                table_names.append(sub_table_name)
                used_variables_by_child.append(sub_used_variables)
                all_used_variables |= sub_used_variables

            except PredicateNotImplementedException as e:
                continue

        if len(table_names) == 0:
            raise PredicateNotImplementedException("All sub-predicates of the and were not implemented")

        if len(table_names) == 1:
            return table_names[0], used_variables_by_child[0]
        
        select_items = [f"{table_names[0]}.trace_id", f"{table_names[0]}.domain"]
        selected_variables = set()
        string_intervals = []
        join_clauses = []

        for i, (sub_table_name, sub_used_variables) in enumerate(zip(table_names, used_variables_by_child)):
            string_intervals.append(f"{sub_table_name}.string_intervals")

            for variable in sub_used_variables:
                if variable not in selected_variables:
                    select_items.append(f'{sub_table_name}."{variable}"')
                    selected_variables.add(variable)
            
            if i > 0:
                join_parts = [f"INNER JOIN {sub_table_name} ON {table_names[0]}.trace_id={sub_table_name}.trace_id"] 

                for j, (prev_table_name, prev_used_variables) in enumerate(zip(table_names[:i], used_variables_by_child[:i])):
                    shared_variables = sub_used_variables & prev_used_variables
                    join_parts.extend([f'{sub_table_name}."{v}"={prev_table_name}."{v}"' for v in shared_variables])

                join_clauses.append(" AND ".join(join_parts))

        
        select_items.append(f'({" & ".join(string_intervals)}) AS string_intervals')

        table_name = self._next_temp_table_name()
        query = f"CREATE TABLE {table_name} AS SELECT {', '.join(select_items)} FROM {table_names[0]} {' '.join(join_clauses)};"
        if DEBUG: print(query)
        duckdb.sql(query)

        self._cleanup_empty_assignments(table_name)        

        return table_name, all_used_variables

    def _cleanup_empty_assignments(self, table_name: str) -> None:
        cleanup_query = f"DELETE FROM {table_name} WHERE bit_count(string_intervals) = 0;"
        if DEBUG: print(cleanup_query)
        duckdb.sql(cleanup_query)

    def _handle_or(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]]) -> typing.Tuple[pl.DataFrame, typing.Set[str]]:
        or_args = predicate["or_args"]
        if not isinstance(or_args, list):
            or_args = [or_args]

        table_names = []
        used_variables_by_child = []
        all_used_variables = set()

        for or_arg in or_args:  # type: ignore
            try:
                sub_table_name, sub_used_variables = self._inner_filter(or_arg, mapping)
                table_names.append(sub_table_name)
                used_variables_by_child.append(sub_used_variables)
                all_used_variables |= sub_used_variables

            except PredicateNotImplementedException as e:
                continue

        if len(table_names) == 0:
            raise PredicateNotImplementedException("All sub-predicates of the por were not implemented")
        
        if len(table_names) == 1:
            return table_names[0], used_variables_by_child[0]

        # Building this table to explicitly represent all potential assignments, instead of implicitly representing them as nulls
        empty_intervals_table_name = self._build_potential_missing_values_table(mapping, list(all_used_variables))

        table_names.insert(0, empty_intervals_table_name)
        used_variables_by_child.insert(0, all_used_variables)

        select_items = [f"{table_names[0]}.trace_id", f"{table_names[0]}.domain"]
        selected_variables = set()
        string_intervals = []
        join_clauses = []

        for i, (sub_table_name, sub_used_variables) in enumerate(zip(table_names, used_variables_by_child)):
            string_intervals.append(f"{sub_table_name}.string_intervals")

            for variable in sub_used_variables:
                if variable not in selected_variables:
                    select_items.append(f'{sub_table_name}."{variable}"')
                    selected_variables.add(variable)
            
            if i > 0:
                join_parts = [f"LEFT JOIN {sub_table_name} ON {table_names[0]}.trace_id={sub_table_name}.trace_id"] 

                shared_variables = sub_used_variables & all_used_variables
                join_parts.extend([f'{sub_table_name}."{v}"={empty_intervals_table_name}."{v}"' for v in shared_variables])

                join_clauses.append(" AND ".join(join_parts))

        string_intervals_coalesce = [f"COALESCE({si}, empty_bitstring())" if i > 0 else si for i, si in enumerate(string_intervals)]
        select_items.append(f'({" | ".join(string_intervals_coalesce)}) AS string_intervals')

        table_name = self._next_temp_table_name()
        query = f"CREATE TABLE {table_name} AS SELECT {', '.join(select_items)} FROM {table_names[0]} {' '.join(join_clauses)};"
        if DEBUG: print(query)
        duckdb.sql(query)

        self._cleanup_empty_assignments(table_name)

        return table_name, all_used_variables
        

    def _build_potential_missing_values_table(self, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]], relevant_vars: typing.List[str]):
        # For each trace ID, and each assignment of the vars that exist in the sub_predicate_df so far:
        relevant_var_mapping = {var: mapping[var] if var.startswith("?") else [var] for var in relevant_vars}
        variable_types = tuple(tuple(relevant_var_mapping[var]) for var in relevant_var_mapping.keys())

        # For each cartesian product of the valid assignments for those vars given the domain
        possible_arg_assignments = [self._object_assignments(domain, variable_types) for domain in self.domains]

        # TODO: if I decide to make the intervals for each trace as long as that trace, I need to revise the logic here to do that, too
        possible_assignments_df = pl.DataFrame(dict(domain=self.domains, assignments=possible_arg_assignments, string_intervals=['0' * self.max_length] * len(self.domains)),
                                                schema=dict(domain=pl.Categorical, assignments=pl.List(pl.List(pl.Categorical)), string_intervals=pl.Utf8))  # type: ignore

        potential_missing_values_df = self.trace_lengths_and_domains_df.join(possible_assignments_df, how="left", on="domain")
        potential_missing_values_df = potential_missing_values_df.explode('assignments').select(
            'domain', 'trace_id', 
            pl.col("assignments").list.to_struct(fields=relevant_vars), 'string_intervals').unnest('assignments')

        table_name = self._next_temp_table_name()
        if DEBUG: print(f"Creating potential missing values table {table_name}")

        assignment_columns = ", ".join(f'"{var}" arg_id' for var in relevant_vars)
        duckdb.sql(f"CREATE TABLE {table_name}(domain domain, trace_id trace_id, {assignment_columns}, string_intervals BITSTRING)");
        duckdb.sql(f"INSERT INTO {table_name} SELECT * FROM potential_missing_values_df;")
        # duckdb.sql(f"ALTER TABLE {table_name} ALTER string_intervals TYPE BITSTRING")
        # for var in relevant_vars:
        #     duckdb.sql(f'ALTER TABLE {table_name} ALTER "{var}" TYPE arg_id')

        return table_name

    def _handle_not(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]]) -> typing.Tuple[pl.DataFrame, typing.Set[str]]:
        try:
            inner_table_name, used_variables = self._inner_filter(predicate["not_args"], mapping)  # type: ignore
        except PredicateNotImplementedException as e:
            raise PredicateNotImplementedException(f"Sub-predicate of the not ({e.args}) was not implemented")


        relevant_vars = list(used_variables)
        potential_missing_values_table_name = self._build_potential_missing_values_table(mapping, relevant_vars)

        # Now, for each possible combination of args on each trace / domain, 'intervals' will contain the truth intervals if
        # they exist and null otherwise, and 'intervals_right' will contain the empty interval'
        join_columns = ["trace_id"] + relevant_vars

        select_items = [f"{potential_missing_values_table_name}.trace_id", f"{potential_missing_values_table_name}.domain"]
        select_items.extend(f"{potential_missing_values_table_name}.\"{var}\"" for var in relevant_vars)
        select_items.append(f"(~( {potential_missing_values_table_name}.string_intervals | COALESCE({inner_table_name}.string_intervals, empty_bitstring()) )) AS string_intervals")
        
        join_items = [f"{potential_missing_values_table_name}.\"{column}\"={inner_table_name}.\"{column}\""  for column in join_columns]

        table_name = self._next_temp_table_name()
        query = f"CREATE TABLE {table_name} AS SELECT {', '.join(select_items)} FROM {potential_missing_values_table_name} LEFT JOIN {inner_table_name} ON {' AND '.join(join_items)};"
        if DEBUG: print(query)
        duckdb.sql(query)

        self._cleanup_empty_assignments(table_name)

        return table_name, used_variables

    def _predicate_and_mapping_cache_key(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]], *args, **kwargs) -> str:
        '''
        Returns a string that uniquely identifies the predicate and mapping
        '''
        return ast_section_to_string(predicate, PREFERENCES) + "_" + str(mapping)


    def _inner_filter(self, predicate: tatsu.ast.AST, mapping: typing.Dict[str, typing.Union[str, typing.List[str]]]) -> typing.Tuple[pl.DataFrame, typing.Set[str]]:
        '''
        Filters the data by the given predicate and mapping, returning a list of intervals in which the predicate is true
        for each processed trace

        Returns a dictionary mapping from the trace ID to a dictionary that maps from the set of arguments to a list of
        intervals in which the predicate is true for that set of arguments
        '''

        predicate_rule = predicate.parseinfo.rule  # type: ignore

        if predicate_rule == "predicate":
            return self._handle_predicate(predicate, mapping)

        elif predicate_rule == "super_predicate":
            return self._inner_filter(predicate["pred"], mapping)  # type: ignore

        elif predicate_rule == "super_predicate_and":
            return self._handle_and(predicate, mapping)

        elif predicate_rule == "super_predicate_or":
            return self._handle_or(predicate, mapping)

        elif predicate_rule == "super_predicate_not":
            return self._handle_not(predicate, mapping)

        elif predicate_rule in ["super_predicate_exists", "super_predicate_forall", "function_comparison"]:
            raise PredicateNotImplementedException(predicate_rule)

        else:
            raise ValueError(f"Error: Unknown rule '{predicate_rule}'")
        

cspsd = CommonSensePredicateStatisticsDatabse()

In [283]:
DEFAULT_GRAMMAR_PATH = "../dsl/dsl.ebnf"
grammar = open(DEFAULT_GRAMMAR_PATH).read()
grammar_parser = typing.cast(tatsu.grammars.Grammar, tatsu.compile(grammar))

game = open(get_project_dir() + '/reward-machine/games/ball_to_bin_from_bed.txt').read()
game_ast = grammar_parser.parse(game)  # type: ignore

block_stacking_game = open(get_project_dir() + '/reward-machine/games/block_stacking.txt').read()
block_stacking_game_ast = grammar_parser.parse(block_stacking_game)  # type: ignore

In [483]:
# test_pred_desk_and = block_stacking_game_ast[3][1]['preferences'][1]['definition']['pref_body']['body']['exists_args']['at_end_pred']
# ast_printer.ast_section_to_string(test_pred_desk_and, ast_parser.PREFERENCES)

test_pred_or = block_stacking_game_ast[3][1]['preferences'][2]['definition']['pref_body']['body']['exists_args']['at_end_pred']
print(ast_printer.ast_section_to_string(test_pred_or, ast_parser.PREFERENCES))

(not  (or    (agent_holds ?b)    (in_motion ?c) ))


In [497]:
cspsd.filter(test_pred_or, {"?b": ["block"], "?c": ["chair"]})

CREATE TEMP TABLE temp_table_1 AS SELECT trace_id, domain, string_intervals, arg_1_id as '?b' FROM data WHERE predicate='agent_holds' AND arg_1_type IN ('bridge_block', 'cube_block', 'cylindrical_block', 'flat_block', 'pyramid_block', 'tall_cylindrical_block', 'tall_rectangular_block', 'triangle_block');
CREATE TEMP TABLE temp_table_2 AS SELECT trace_id, domain, string_intervals, arg_1_id as '?c' FROM data WHERE predicate='in_motion' AND arg_1_type='chair';
Creating potential missing values table temp_table_3
CREATE TABLE temp_table_4 AS SELECT temp_table_3.trace_id, temp_table_3.domain, temp_table_3."?b", temp_table_3."?c", (temp_table_3.string_intervals | COALESCE(temp_table_1.string_intervals, empty_bitstring()) | COALESCE(temp_table_2.string_intervals, empty_bitstring())) AS string_intervals FROM temp_table_3 LEFT JOIN temp_table_1 ON temp_table_3.trace_id=temp_table_1.trace_id AND temp_table_1."?b"=temp_table_3."?b" LEFT JOIN temp_table_2 ON temp_table_3.trace_id=temp_table_2.trac

(382, 10236, 359706286)

359706286

In [475]:
q = """SELECT temp_table_3.trace_id, temp_table_3.domain, temp_table_3."top_drawer", temp_table_3."?g", 
(temp_table_3.string_intervals | COALESCE(temp_table_1.string_intervals, empty_bitstring()) | COALESCE(temp_table_2.string_intervals, empty_bitstring())) AS si,
FROM temp_table_3 
LEFT JOIN temp_table_1 ON temp_table_3.trace_id=temp_table_1.trace_id AND temp_table_1."top_drawer"=temp_table_3."top_drawer" AND temp_table_1."?g"=temp_table_3."?g" 
LEFT JOIN temp_table_2 ON temp_table_3.trace_id=temp_table_2.trace_id AND temp_table_2."top_drawer"=temp_table_3."top_drawer" 
"""

# temp_table_3.string_intervals as si3, COALESCE(temp_table_1.string_intervals, empty_bitstring()) as si1, COALESCE(temp_table_2.string_intervals, empty_bitstring()) AS si2,
# 
q2 = """SELECT temp_table_3.trace_id, temp_table_3.domain, temp_table_3."top_drawer", temp_table_3."?g", 
(temp_table_3.string_intervals | COALESCE(temp_table_1.string_intervals, empty_bitstring()) | COALESCE(temp_table_2.string_intervals, empty_bitstring())) AS si,
FROM temp_table_3 
LEFT JOIN temp_table_1 ON temp_table_3.trace_id=temp_table_1.trace_id AND temp_table_1."top_drawer"=temp_table_3."top_drawer" AND temp_table_1."?g"=temp_table_3."?g" 
LEFT JOIN temp_table_2 ON temp_table_3.trace_id=temp_table_2.trace_id AND temp_table_2."top_drawer"=temp_table_3."top_drawer"
"""

In [509]:
pldf = pl.from_dataframe(split_args_df)

NotImplementedError: Non-string object dtypes are not supported yet

['temp_table_1', 'temp_table_2']

In [158]:
# duckdb.sql("CREATE TYPE predicate AS ENUM (SELECT predicate FROM split_args_df);")
# duckdb.sql("CREATE TYPE domain AS ENUM (SELECT domain FROM split_args_df);")
# duckdb.sql("CREATE TYPE trace_id AS ENUM (SELECT trace_id FROM split_args_df);")
# all_types = tuple([t for t in set(split_args_df.arg_1_type.unique()) | set(split_args_df.arg_2_type.unique()) if isinstance(t, str) ])
# duckdb.sql(f"CREATE TYPE arg_type AS ENUM {all_types};")
# all_ids = tuple([t for t in set(split_args_df.arg_1_id.unique()) | set(split_args_df.arg_2_id.unique()) if isinstance(t, str)])
# duckdb.sql(f"CREATE TYPE arg_id AS ENUM {all_ids};")

# duckdb.sql('ALTER TABLE test ALTER predicate TYPE predicate')
# duckdb.sql('ALTER TABLE test ALTER domain TYPE domain')
# duckdb.sql('ALTER TABLE test ALTER trace_id TYPE trace_id')
# duckdb.sql('ALTER TABLE test ALTER arg_1_type TYPE arg_type')
# duckdb.sql('ALTER TABLE test ALTER arg_2_type TYPE arg_type')
# duckdb.sql('ALTER TABLE test ALTER arg_1_id TYPE arg_id')
# duckdb.sql('ALTER TABLE test ALTER arg_2_id TYPE arg_id')

In [161]:
duckdb.sql('DESCRIBE test').show()

┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│   column_name    │ column_type │  null   │   key   │ default │ extra │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ predicate        │ predicate   │ YES     │ NULL    │ NULL    │  NULL │
│ arg_1_id         │ arg_id      │ YES     │ NULL    │ NULL    │  NULL │
│ arg_1_type       │ arg_type    │ YES     │ NULL    │ NULL    │  NULL │
│ arg_2_id         │ arg_id      │ YES     │ NULL    │ NULL    │  NULL │
│ arg_2_type       │ arg_type    │ YES     │ NULL    │ NULL    │  NULL │
│ trace_id         │ trace_id    │ YES     │ NULL    │ NULL    │  NULL │
│ domain           │ "domain"    │ YES     │ NULL    │ NULL    │  NULL │
│ intervals        │ INTEGER[][] │ YES     │ NULL    │ NULL    │  NULL │
│ string_intervals │ BIT         │ YES     │ NULL    │ NULL    │  NULL │
└──────────────────┴─────────────┴─────────┴───────

In [106]:
duckdb.sql("SELECT bit_count((SELECT string_intervals FROM test WHERE predicate='on' LIMIT 1) & (SELECT string_intervals FROM test WHERE predicate='on' OFFSET 100 LIMIT 1))")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ bit_count(((SELECT string_intervals FROM test WHERE (predicate = 'on') LIMIT 1) & (SELECT string_intervals FROM te…  │
│                                                        int64                                                         │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                                                  675 │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [88]:
duckdb.sql("SELECT intervals FROM test WHERE predicate='on' LIMIT 1")

┌─────────────┐
│  intervals  │
│  int16[][]  │
├─────────────┤
│ [[0, 1612]] │
└─────────────┘

In [501]:
arg_type = 'basketball'
duckdb.sql(f"SELECT count(*) FROM data WHERE (arg_1_type='{arg_type}' OR arg_2_type='{arg_type}')").fetchone()[0]

5802

In [503]:
duckdb.sql("SELECT val FROM (SELECT unnest(enum_range(NULL::arg_id)) as val) ")  # WHERE val='Shelf|-02.97|+01.53|-01.72' 

┌────────────────────────────────────┐
│                val                 │
│              varchar               │
├────────────────────────────────────┤
│ south_wall                         │
│ building_27                        │
│ CylinderBlock|-02.97|+01.62|-01.50 │
│ Shelf|+03.13|+00.63|-00.56         │
│ building_11                        │
│ PyramidBlock|-02.96|+01.61|-02.44  │
│ Mirror|+00.45|+01.49|+00.62        │
│ Pencil|+03.07|+00.79|-01.79        │
│ building_9                         │
│ PyramidBlock|-02.95|+01.61|-02.66  │
│                ·                   │
│                ·                   │
│                ·                   │
│ BridgeBlock|-02.92|+00.26|-02.52   │
│ TeddyBear|-02.60|+00.60|-00.42     │
│ PyramidBlock|+01.13|+01.78|-02.89  │
│ building_12                        │
│ AlarmClock|-01.41|+00.60|+00.45    │
│ Book|+02.83|+00.41|-00.01          │
│ CylinderBlock|+00.93|+01.61|-02.89 │
│ building_8                         │
│ Drawer|-01.52|+00.14|+0

In [ ]:
bits_df.select(pl.col('bits_1') & pl.col('bits_2'))

In [ ]:
all_specific_names = set()
for room, type_to_ids in SPECIFIC_NAMED_OBJECTS_BY_ROOM.items():
    all_specific_names.update(type_to_ids.keys())

specific_name_rows = split_args_df[(split_args_df.arg_1_type.isin(all_specific_names) | split_args_df.arg_2_type.isin(all_specific_names))].shape[0]
total_rows = split_args_df.shape[0]
print(f'{specific_name_rows} / {total_rows} ({specific_name_rows / total_rows * 100:.2f}%) rows have a specific name')

In [ ]:
same_type_rows = split_args_df[(split_args_df.predicate == 'same_type')].shape[0]
print(f'{same_type_rows} / {total_rows} ({same_type_rows / total_rows * 100:.2f}%) rows are for same_type')

In [ ]:
for i, row in split_args_df.iterrows():
    print(i, row.to_dict())
    break

In [ ]:
split_args_df[(split_args_df.predicate == 'adjacent') & (split_args_df.arg_1_type == 'agent') & (split_args_df.arg_2_type == 'green_golfball')]

In [ ]:
split_args_df[(split_args_df.predicate == 'same_type')]

array(['medium', 'medium', 'medium', ..., 'many', 'many', 'many'],
      dtype=object)

In [ ]:
door_df = split_args_df[(split_args_df.arg_1_type == 'door') | (split_args_df.arg_1_type == 'door')]
print(door_df.predicate.value_counts())
print()
for predicate in door_df.predicate.unique():
    pred_df = door_df[door_df.predicate == predicate]
    print(f'For predicate {predicate}:')
    print(pred_df.arg_2_type.value_counts().iloc[:10])
    print()

In [ ]:
for tid in door_df[door_df.predicate == 'in_motion'].trace_id:
    print(tid)

In [ ]:
split_args_df[split_args_df.predicate == 'in_motion'].arg_1_type.value_counts()

In [ ]:
stats._invert_intervals(regular_df[(regular_df.trace_id == trace_id) & (regular_df.predicate == 'in_motion') & (regular_df.arg_ids == (ball_id,))].intervals.values[0],
                        stats.trace_lengths[trace_id])

In [ ]:
split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in') & (split_args_df.arg_1_id == bin_id) & (split_args_df.arg_2_id == ball_id)]

In [ ]:
ball_not_in_motion_int = split_args_stats._invert_intervals(
    split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in_motion') & (split_args_df.arg_1_id == ball_id)].intervals.values[0],
    split_args_stats.trace_lengths_and_domains[trace_id][0],
)

ball_not_in_motion_int

In [ ]:
ball_in_bin_int = split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in') & (split_args_df.arg_1_id == bin_id) & (split_args_df.arg_2_id == ball_id)].intervals.values[0]

split_args_stats._intersect_intervals(ball_in_bin_int, ball_not_in_motion_int)

In [ ]:
single_trace_split_args_df = split_args_df[split_args_df['trace_id'] == trace_id]

single_trace_split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(cache_dir)
single_trace_split_args_stats.data = pl.from_pandas(single_trace_split_args_df)
single_trace_split_args_stats.data.shape

In [ ]:
DEFAULT_GRAMMAR_PATH = "../dsl/dsl.ebnf"
grammar = open(DEFAULT_GRAMMAR_PATH).read()
grammar_parser = typing.cast(tatsu.grammars.Grammar, tatsu.compile(grammar))

game = open(compile_predicate_statistics.get_project_dir() + '/reward-machine/games/ball_to_bin_from_bed.txt').read()
game_ast = grammar_parser.parse(game) 

test_pred_1 = game_ast[4][1]['preferences'][0]['definition']['forall_pref']['preferences']['pref_body']['body']['exists_args']['then_funcs'][1]['seq_func']['hold_pred']

# should be: (and (not (in_motion ?b)) (in ?h ?b)))
test_pred_2 = game_ast[4][1]['preferences'][0]['definition']['forall_pref']['preferences']['pref_body']['body']['exists_args']['then_funcs'][2]['seq_func']['once_pred']

In [ ]:
test_mapping = {"?b": ["ball"], "?h": ["hexagonal_bin"]}
test_out = single_trace_split_args_stats.filter(test_pred_2, test_mapping)
test_out